<a href="https://colab.research.google.com/github/kirbu123/FilmGeneration/blob/develop/FilmGenerationAttemp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import usefull libs

In [120]:
import torch
import time
from datetime import datetime
from tqdm import tqdm
import pandas as pd
from google.colab import drive
import os
import math
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE
import numpy as np
from sklearn import svm
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, f1_score
from sklearn.model_selection import cross_val_score, train_test_split, RepeatedKFold, KFold
from sklearn.multiclass import OneVsRestClassifier
from PIL import Image
import requests
from io import BytesIO
from bs4 import BeautifulSoup
import pandas as pd
import csv
from urllib.request import urlopen

## Load dataset

Go to root

In [6]:
cd /content

/content


Make data directory, go there

In [7]:
mkdir laion-high-resolution && cd laion-high-resolution

In [11]:
# Function for loading 100 rows of dataset from the url
# Param filtering indicate filtering of data by languages (langs) when it is filtering = True
# Param filter_lang sorts data from all langs except filter_lang
# Param file_name sets file_name + .csv
def LoadLaionDataset(filtering = False, filter_lang = '', load_csv = False, file_name = 'dataset', url = 'https://huggingface.co/datasets/laion/laion-high-resolution/viewer/default/train'):
  # Created an URL object in url
  # Create object page
  page = requests.get(url)
  # parser-lxml = Change html to Python friendly format
  # Obtain page's information
  soup = BeautifulSoup(page.text, 'lxml')
  # Obtain information from tag <table>
  table = soup.find('table', attrs={"class": "w-full table-auto rounded-lg font-mono text-xs text-gray-900"})
  # Obtain every title of columns with tag <th>
  headers = []
  for i in table.find_all('th'):
    title = i.text
    headers.append(title[:title.find('\n')])
  # Create a dataframe
  df = pd.DataFrame(columns = headers)
  # Create a for loop to fill mydata
  for j in table.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [i.text[:i.text.find('\n')] for i in row_data]
    for i in range(len(row)):
      row[i] = row[i][row[i].find('"') + 1: row[i].rfind('"')]
    length = len(df)
    df.loc[length] = row
  # Filtering by languages
  if filtering:
    df = df[df['LANGUAGE'] == filter_lang]
  # Loading .csv
  if load_csv:
    df.to_csv ('/content/laion-high-resolution/' + str(file_name) + '.csv', index= False )
  return df

In [12]:
df = LoadLaionDataset(load_csv = True)

In [13]:
df.head()

,URL,TEXT,WIDTH,HEIGHT,similarity,LANGUAGE,hash,pwatermark,punsafe
0,https://mmedia.ozone.ru/multimedia/1018085734.JPG,"Штора рулонная Эскар, цвет: абрикосовый, ширин...","1,02","1,20",0.27589,ru,"-3,448,109,611,037,364,00",0.05275,0.00129
1,https://images.squarespace-cdn.com/content/v1/...,Elysia Lounge chair from DeLaEspada,"2,50","1,66",0.29183,nolang,"1,517,529,356,012,770,30",0.05878,0.00001
2,https://images.footballfanatics.com/FFImage/th...,Houston Texans Striped Prime Time Backpack,"2,00","2,00",0.39709,en,"5,924,454,315,519,412,00",nul,nul
3,https://images.squarespace-cdn.com/content/v1/...,©Radiohead-Bandcamp,"1,20","1,20",0.26665,nolang,"-3,933,968,474,534,865,40",0.06562,0.00001
4,"https://s3.r29static.com/bin/entry/4ce/0,0,200...",Amazon Registry Best Wedding Gift Items,"2,00","1,05",0.31002,en,"7,893,281,056,273,843,00",0.15549,0.00004


Upgrade of dataset loading function

In [123]:
# Param frames_count sets count of loading url pages
# Other params are same eith previous function
def LoadData(frames_count = 1, filtering = False, filter_lang = '', load_csv = False, file_name = 'dataset', based_url = 'https://huggingface.co/datasets/laion/laion-high-resolution/viewer/default/train'):
  # DataFrame list
  df_list = list()
  # Updating df_list by loading .csv by url
  for i in tqdm(range(frames_count)):
    df_list.append(LoadLaionDataset(filtering, filter_lang, load_csv = False, url = based_url + '?p=' + str(i)))
  # concating df_list to one big DataFrame
  result_df = pd.concat(df_list)
  X = result_df[['TEXT', 'LANGUAGE']].to_numpy()
  y = result_df['URL'].to_numpy()
  return result_df, list([X, y])

In [124]:
df, dataset = LoadData(frames_count = 2)

100%|██████████| 2/2 [00:07<00:00,  3.83s/it]


In [125]:
len(df)

200

In [126]:
df.head()

,URL,TEXT,WIDTH,HEIGHT,similarity,LANGUAGE,hash,pwatermark,punsafe
0,https://mmedia.ozone.ru/multimedia/1018085734.JPG,"Штора рулонная Эскар, цвет: абрикосовый, ширин...","1,02","1,20",0.27589,ru,"-3,448,109,611,037,364,00",0.05275,0.00129
1,https://images.squarespace-cdn.com/content/v1/...,Elysia Lounge chair from DeLaEspada,"2,50","1,66",0.29183,nolang,"1,517,529,356,012,770,30",0.05878,0.00001
2,https://images.footballfanatics.com/FFImage/th...,Houston Texans Striped Prime Time Backpack,"2,00","2,00",0.39709,en,"5,924,454,315,519,412,00",nul,nul
3,https://images.squarespace-cdn.com/content/v1/...,©Radiohead-Bandcamp,"1,20","1,20",0.26665,nolang,"-3,933,968,474,534,865,40",0.06562,0.00001
4,"https://s3.r29static.com/bin/entry/4ce/0,0,200...",Amazon Registry Best Wedding Gift Items,"2,00","1,05",0.31002,en,"7,893,281,056,273,843,00",0.15549,0.00004


In [127]:
dataset[0][0]

array(['Штора рулонная Эскар, цвет: абрикосовый, ширина 140 см, высота 170 см81012140170Рулонными шторами Эскар можно оформлять окна как самостоятельно, так и использовать в комбинации с портьерами. Это поможет предотвратить выгорание дорогой ткани на солнце и соединит функционал рулонных с красотой навесных. Преимущества применения рулонных штор для пластиковых окон: - имеют прекрасный внешний вид: многообразие и фактурность материала изделия отлично смотрятся в любом интерьере;- многофункциональны: есть возможность подобрать шторы способные эффективно защитить комнату от солнца, при этом она не будет слишком темной;- есть возможность осуществить быстрый монтаж.ВНИМАНИЕ! Размеры ширины изделия указаны по ширине ткани! Во время эксплуатации не рекомендуется полностью разматывать рулон, чтобы не оторвать ткань от намоточного вала. В случае загрязнения поверхности ткани, чистку шторы проводят одним из способов, в зависимости от типа загрязнения:легкое поверхностное загрязнение можно удал

## KFold sampling on train-test split

In [129]:
# This func makes KFold from Sklearn more convinient to out task
def KFoldSampling(dataset, nsplits):
  # X, y split from dataset
  X, y = dataset[0], dataset[1]
  kf = KFold(n_splits = nsplits)
  kf.get_n_splits()
  # list of indicies of KFold
  result = list()
  for i, (train_index, test_index) in tqdm(enumerate(kf.split(X))):
    # Appending train-test indicies ro result list
    result.append(np.array([train_index, test_index]))
  return np.array(result)

In [130]:
KFoldSampling(dataset, nsplits=2)

2it [00:00, 3721.65it/s]


array([[[100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
         112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123,
         124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135,
         136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
         148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159,
         160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171,
         172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
         184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195,
         196, 197, 198, 199],
        [  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
          12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,
          24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,
          36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,
          48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,
          60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,
    

## Dataset transform from url to image

In [149]:
def DatasetTransform(dataset, time_limit = 20):
  # Making X, y from dataset
  X = dataset[0]
  y = dataset[1]
  # Making result lists
  result_y = list()
  result_X = list()
  # Start time
  start_time = datetime.now()
  for i, url in enumerate(tqdm(y)):
    # All exceptions are ignored
    try:
      # Load image
      response = requests.get(url)
      img = Image.open(BytesIO(response.content))
    except Exception:
      continue
    # Appending numpy image
    result_y.append(np.array(img))
    result_X.append(X[i])
    # Checking time break
    if (datetime.now() - start_time).seconds > time_limit:
      print('Time break')
      break
  return list([np.array(result_X), np.array(result_y)])

In [150]:
DatasetTransform(dataset)

 12%|█▏        | 24/200 [00:21<02:38,  1.11it/s]

Time break



<ipython-input-149-cbfd6cee3821>:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return list([np.array(result_X), np.array(result_y)])


[array([['Штора рулонная Эскар, цвет: абрикосовый, ширина 140 см, высота 170 см81012140170Рулонными шторами Эскар можно оформлять окна как самостоятельно, так и использовать в комбинации с портьерами. Это поможет предотвратить выгорание дорогой ткани на солнце и соединит функционал рулонных с красотой навесных. Преимущества применения рулонных штор для пластиковых окон: - имеют прекрасный внешний вид: многообразие и фактурность материала изделия отлично смотрятся в любом интерьере;- многофункциональны: есть возможность подобрать шторы способные эффективно защитить комнату от солнца, при этом она не будет слишком темной;- есть возможность осуществить быстрый монтаж.ВНИМАНИЕ! Размеры ширины изделия указаны по ширине ткани! Во время эксплуатации не рекомендуется полностью разматывать рулон, чтобы не оторвать ткань от намоточного вала. В случае загрязнения поверхности ткани, чистку шторы проводят одним из способов, в зависимости от типа загрязнения:легкое поверхностное загрязнение можно уд